In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import warnings

In [3]:
low_memory=False

In [5]:
warnings.filterwarnings('ignore')

In [13]:
df=pd.read_csv('bronzelayer_Spotify_Charts.csv')

In [11]:
# EDA to understand patterns within songs dataset
df['title'] = df['title'].str.strip().str.lower()
exact_duplicates = df['title'][df['title'].duplicated(keep=False)].value_counts()
if exact_duplicates.empty:
    print("None")
else:
    print("Exact duplicate values in 'title' column:")
    for title, count in exact_duplicates.items():
        print(f"'{title}' appears {count} times")

Exact duplicate values in 'title' column:
'shape of you' appears 69041 times
'believer' appears 60176 times
'happier' appears 59632 times
'someone you loved' appears 57595 times
'dance monkey' appears 56923 times
'blinding lights' appears 56058 times
'perfect' appears 55433 times
'shallow' appears 48174 times
'bad guy' appears 45388 times
'don't start now' appears 44653 times
'sunflower - spider-man: into the spider-verse' appears 43508 times
'señorita' appears 43037 times
'falling' appears 41810 times
'lucid dreams' appears 41592 times
'watermelon sugar' appears 41473 times
'lovely (with khalid)' appears 40932 times
'goosebumps' appears 40354 times
'roses - imanbek remix' appears 40157 times
'thunder' appears 40078 times
'say you won't let go' appears 39153 times
'without me' appears 37448 times
'closer' appears 36275 times
'something just like this' appears 36232 times
'new rules' appears 35527 times
'sicko mode' appears 34145 times
'memories' appears 33792 times
'sad!' appears 33686

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
# Display unique values in the 'trend' column
unique_trends = df['trend'].unique()

print("Unique values in 'trend' column:")
print(unique_trends)

Unique values in 'trend' column:
['SAME_POSITION' 'MOVE_UP' 'MOVE_DOWN' 'NEW_ENTRY']


In [11]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncode smaller categorical columns
enc = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = enc.fit_transform(chunk[['chart', 'trend']])


In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

# Initialize model for incremental learning
model = LinearRegression()

# Initialize Label Encoders for high cardinality columns
label_enc_title = LabelEncoder()
label_enc_artist = LabelEncoder()
label_enc_region = LabelEncoder()

# Set the total number of chunks (optional, if you know the total size of the dataset)
total_chunks = 10  # Replace with the actual number if known, or calculate dynamically

# Load data in chunks
chunksize = 1000000  # Adjust chunk size to lower memory usage
chunks = []
chunk_num = 0  # To keep track of the chunk number

for chunk in pd.read_csv('bronzelayer_Spotify_Charts.csv', chunksize=chunksize):
    chunk_num += 1
    print(f"Processing chunk {chunk_num}")

    # Preprocess the data
    chunk['song_id'] = chunk['title'] + '_' + chunk['artist'] + '_' + chunk['region']

    # Label encode high cardinality columns (title, artist, region)
    chunk['title'] = label_enc_title.fit_transform(chunk['title'])
    chunk['artist'] = label_enc_artist.fit_transform(chunk['artist'])
    chunk['region'] = label_enc_region.fit_transform(chunk['region'])

    # Selecting features (X) and target (y)
    X = chunk[['title', 'artist', 'region', 'chart', 'trend', 'year', 'month', 'day_of_week']]
    y = chunk['streams']

    # Fill missing 'streams' temporarily with 0
    y.fillna(0, inplace=True)

    # OneHotEncode smaller categorical columns
    enc = OneHotEncoder(drop='first', sparse_output=False)
    X_encoded = enc.fit_transform(chunk[['chart', 'trend']])

    # Combine label encoded and one-hot encoded features
    X_combined = np.hstack([chunk[['title', 'artist', 'region', 'year', 'month', 'day_of_week']].values, X_encoded])

    # Train the regression model on each chunk
    model.fit(X_combined, y)

    # Predict the 'streams' values
    chunk['estimated_streams'] = model.predict(X_combined)
    
    # Adjust the estimated 'streams' based on the 'trend'
    def adjust_by_trend(row):
        if row['trend'] == 'Up':
            return row['estimated_streams'] * 1.1  # Increase by 10%
        elif row['trend'] == 'Down':
            return row['estimated_streams'] * 0.9  # Decrease by 10%
        else:
            return row['estimated_streams']  # No change for 'Same'
    
    # Apply the trend adjustment
    chunk['adjusted_streams'] = chunk.apply(adjust_by_trend, axis=1)
     
    # Fill in the missing 'streams' values using the adjusted estimates
    chunk['streams'] = chunk['streams'].fillna(chunk['adjusted_streams'])

    chunks.append(chunk)

# Concatenate the processed chunks
df = pd.concat(chunks, ignore_index=True)

# Save the processed data to a 'silver layer' CSV file
df.to_csv('spotify_charts_silver.csv', index=False)
print("Processed dataset saved to 'spotify_charts_silver.csv'")


Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Processing chunk 16
Processing chunk 17
Processing chunk 18
Processing chunk 19
Processing chunk 20
Processing chunk 21
Processing chunk 22
Processing chunk 23
Processing chunk 24
Processing chunk 25
Processing chunk 26
Processing chunk 27
Processed dataset saved to 'spotify_charts_silver.csv'


In [9]:
# Show rows where streams' column has null values
df2=pd.read_csv('spotify_charts_silver.csv')
null_stream_rows = df2[df2['streams'].isnull()]

print(null_stream_rows)

Empty DataFrame
Columns: [title, rank, date, artist, url, region, chart, trend, streams, year, month, day, day_of_week, song_id, estimated_streams, adjusted_streams]
Index: []


In [13]:
df2.head()

,title,rank,date,artist,url,region,chart,trend,streams,year,month,day,day_of_week,song_id,estimated_streams,adjusted_streams
0,3137,1,2017-01-01,10688,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1,top200,SAME_POSITION,253019.0,2017,1,1,6,Chantaje (feat. Maluma)_Shakira_Argentina,92528.837086,92528.837086
1,18674,2,2017-01-01,9982,https://open.spotify.com/track/7DM4BPaS7uofFul...,1,top200,MOVE_UP,223988.0,2017,1,1,6,Vente Pa' Ca (feat. Maluma)_Ricky Martin_Argen...,43476.243022,43476.243022
2,14526,3,2017-01-01,1856,https://open.spotify.com/track/3AEZUABDXNtecAO...,1,top200,MOVE_DOWN,210943.0,2017,1,1,6,Reggaetón Lento (Bailemos)_CNCO_Argentina,42769.585145,42769.585145
3,15047,4,2017-01-01,5115,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1,top200,SAME_POSITION,173865.0,2017,1,1,6,"Safari_J Balvin, Pharrell Williams, BIA, Sky_A...",88552.836659,88552.836659
4,15599,5,2017-01-01,2584,https://open.spotify.com/track/58IL315gMSTD37D...,1,top200,MOVE_UP,153956.0,2017,1,1,6,Shaky Shaky_Daddy Yankee_Argentina,43133.315269,43133.315269


In [13]:
# In the previous machine learning model we also encoded columns artist and title as they are highly important to determine number of streams 
# By encoding them we lose names of artists and song titles, now we reverse this action by replacing them to save the dataset properly to be used in the gold layer.
original_df = pd.read_csv('bronzelayer_Spotify_Charts.csv')
encoded_df = df2

# Add the 'artist' and 'title' columns from the original dataset to the encoded dataset
df2['artist'] = original_df['artist']
df2['title'] = original_df['title']

encoded_df.to_csv('spotify_charts_silver_with_artists_and_titles.csv', index=False)
print("Merged dataset saved as 'spotify_charts_silver_with_artists_and_titles.csv'")

Merged dataset saved as 'spotify_charts_silver_with_artists_and_titles.csv'


In [7]:
df=pd.read_csv('spotify_charts_silver_with_artists_and_titles.csv')
df.head()

,title,rank,date,artist,url,region,chart,trend,streams,year,month,day,day_of_week,song_id,estimated_streams,adjusted_streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1,top200,SAME_POSITION,253019.0,2017,1,1,6,Chantaje (feat. Maluma)_Shakira_Argentina,92528.837086,92528.837086
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,1,top200,MOVE_UP,223988.0,2017,1,1,6,Vente Pa' Ca (feat. Maluma)_Ricky Martin_Argen...,43476.243022,43476.243022
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,1,top200,MOVE_DOWN,210943.0,2017,1,1,6,Reggaetón Lento (Bailemos)_CNCO_Argentina,42769.585145,42769.585145
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1,top200,SAME_POSITION,173865.0,2017,1,1,6,"Safari_J Balvin, Pharrell Williams, BIA, Sky_A...",88552.836659,88552.836659
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,1,top200,MOVE_UP,153956.0,2017,1,1,6,Shaky Shaky_Daddy Yankee_Argentina,43133.315269,43133.315269
